### Pobranie datasetu MVTEC

In [ ]:
from mvtec import MVTecSingleClassDataset, CLASS_NAMES
import os

print("MVTec-AD classes:")
for i,class_name in enumerate(CLASS_NAMES):
    print(f"{i+1}. {class_name}")


In [ ]:
# Wybór klasy z datasetu MVTec-AD
CLASS = "bottle"

In [ ]:
# Pobieranie jedynie wybranej klasy z datasetu

DATASET_PATH = os.path.abspath("dataset")
dataset = MVTecSingleClassDataset(dataset_path=DATASET_PATH, class_name=CLASS)

print(f"Number of images: {len(dataset)}")

In [ ]:
# Przegląd danych

import matplotlib.pyplot as plt
import glob
import cv2 as cv
from mvtec import TRANSFORM
from PIL import Image
import numpy as np

def get_transform_visualisation(image):
    image = Image.fromarray(image)
    image = TRANSFORM(image).numpy()
    image = np.moveaxis(image, 0, -1)
    # Normalize to 0->1
    image = (image - image.min()) / (image.max() - image.min())

    return image

for anomaly_type in os.listdir(f"{DATASET_PATH}/{CLASS}/test"):
    im = glob.glob(f"{DATASET_PATH}/{CLASS}/test/{anomaly_type}/*.png")[0]
    image = cv.imread(im)
    mask_path = f"{DATASET_PATH}/{CLASS}/ground_truth/{anomaly_type}/{os.path.basename(im).split('.')[0]}_mask.png"
    
    
    if os.path.exists(mask_path):
        mask = cv.imread(mask_path,cv.IMREAD_GRAYSCALE)
        contour = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)[0]
        cv.drawContours(image, contour, -1, (255, 0, 0), 2)
    
    image_transformed = get_transform_visualisation(image)
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    image_transformed = cv.cvtColor(image_transformed, cv.COLOR_BGR2RGB)
    
    fig,axes = plt.subplots(1,2,figsize=(10,5))
    
    axes[0].imshow(image)
    axes[1].imshow(image_transformed)

    axes[0].set_title(anomaly_type.capitalize())
    axes[1].set_title(anomaly_type.capitalize()+" (transformed)")

    axes[0].axis("off")
    axes[1].axis("off")
    
    fig.tight_layout()
    plt.show()


### Definicja subsetów train oraz test

In [ ]:
train_dataset = MVTecSingleClassDataset(DATASET_PATH, class_name=CLASS, subset='train')
test_dataset = MVTecSingleClassDataset(DATASET_PATH, class_name=CLASS, subset='test')

In [ ]:
from padim import Padim
import numpy as np

ARCHITECTURE = 'wide_resnet50_2' #'resnet18'

padim = Padim(arch=ARCHITECTURE, save_path='results', classname=CLASS)

In [ ]:
if not os.path.exists(f'results/embedding_{train_dataset.class_name}.npz'):
    embedding_vectors_test, anomaly_types_test = padim.extract_features(test_dataset)

    embedding_vectors_test = embedding_vectors_test.cpu().numpy()

    np.savez_compressed(f'results/embedding_{test_dataset.class_name}.npz', embedding_vectors=embedding_vectors_test, anomaly_types=anomaly_types_test)
else:
    data = np.load(f'results/embedding_{test_dataset.class_name}.npz')
    embedding_vectors_test = data["embedding_vectors"]
    anomaly_types_test = data["anomaly_types"]

    del data


print(embedding_vectors_test.shape)
embedding_vectors_test = np.max(embedding_vectors_test, axis=(2,3))
print(embedding_vectors_test.shape)

In [ ]:
from matplotlib import pyplot as plt
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, random_state=0)
embedding_vectors_tsne = tsne.fit_transform(embedding_vectors_test)

plt.figure(figsize=(6, 5))
plt.scatter(embedding_vectors_tsne[anomaly_types_test == "good", 0], embedding_vectors_tsne[anomaly_types_test == "good", 1], label="good", marker='.', color='black')
for anomaly_type in np.unique(anomaly_types_test):
    if anomaly_type == "good": continue
    plt.scatter(embedding_vectors_tsne[anomaly_types_test == anomaly_type, 0], embedding_vectors_tsne[anomaly_types_test == anomaly_type, 1], label=anomaly_type, marker='x')
plt.legend()
plt.title('TSNE')
plt.xlabel('TSNE 1')
plt.ylabel('TSNE 2')
plt.tight_layout()
plt.show()

del embedding_vectors_tsne

In [ ]:
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA

pca = PCA(n_components=2, random_state=0)
embedding_vectors_pca = pca.fit_transform(embedding_vectors_test)
pca.explained_variance_ratio_

plt.figure(figsize=(6, 5))
plt.scatter(embedding_vectors_pca[anomaly_types_test == "good", 0], embedding_vectors_pca[anomaly_types_test == "good", 1], label="good", marker='.', color='black')
for anomaly_type in np.unique(anomaly_types_test):
    if anomaly_type == "good": continue
    plt.scatter(embedding_vectors_pca[anomaly_types_test == anomaly_type, 0], embedding_vectors_pca[anomaly_types_test == anomaly_type, 1], label=anomaly_type, marker='x')
plt.legend()
plt.title('PCA')
plt.xlabel('PC 1: {:.2f}%'.format(pca.explained_variance_ratio_[0]*100))
plt.ylabel('PC 2: {:.2f}%'.format(pca.explained_variance_ratio_[1]*100))
plt.tight_layout()
plt.show()

del embedding_vectors_pca

In [ ]:
padim.train(train_dataset=train_dataset)

In [ ]:
padim.validate(test_dataset=test_dataset)

In [ ]:
import glob
import cv2 as cv
from time import time
import random

padim.threshold = 0.55

image_sample = random.sample(glob.glob(f'{DATASET_PATH}\\{CLASS}\\test\\*\\*.png'), k=5)

for im_path in image_sample:
    print(im_path)

    image = cv.imread(im_path)
    class_type = im_path.split("\\")[-2]
    processing_time = time()
    mask, heatmap = padim.preview(image)
    processing_time = time() - processing_time


    mask = cv.resize(mask, (image.shape[1], image.shape[0]), cv.INTER_NEAREST_EXACT).astype(np.uint8)
    heatmap = cv.resize(heatmap, (image.shape[1], image.shape[0]), cv.INTER_CUBIC).astype(np.uint8)    
    heatmap = cv.applyColorMap(heatmap, cv.COLORMAP_JET)

    cv.addWeighted(heatmap, 0.5, image, 0.5, 0, image)
    contours = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)[0]
    
    cv.drawContours(image, contours, -1, (0,255,0), 2)

    mask_path = f"{os.path.dirname(os.path.dirname(os.path.dirname(im_path)))}\\ground_truth\\{os.path.basename(os.path.dirname(im_path))}\\{os.path.basename(im_path).split('.')[0]}_mask.png"
    if os.path.exists(mask_path):
        mask_gt = cv.imread(mask_path)
    else:
        mask_gt = np.zeros_like(image)
    cv.drawContours(mask_gt, contours, -1, (255,0,0), 2)
    fig,axes = plt.subplots(1,2,figsize=(10,5))
    axes[0].imshow(cv.cvtColor(image, cv.COLOR_BGR2RGB))
    axes[0].axis("off")

    axes[1].imshow(mask_gt, cmap='gray')
    axes[1].axis("off")

    fig.suptitle(f"{class_type.capitalize()} - {os.path.basename(im_path)}. Processing time: {processing_time:.1f} s")

    fig.tight_layout()
    plt.show()